In [ ]:
# We're using these tools to help us work with data and make predictions
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

# Let's open our data file and read it
df = pd.read_stata('C:/dhs_zm/ZMBR71DT_birth recode/ZMBR71FL.DTA', convert_categoricals=False)